In [1]:
from glob import glob
import pandas as pd
from scipy.interpolate import interp1d
from scipy.signal import convolve2d
import numpy as np
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment

from lavian_et_al_2025.imaging.imaging_classes import LightsheetExperiment, TwoPExperiment
from lavian_et_al_2025.visual_motion.stimulus_functions import stim_vel_dir_dataframe, quantize_directions

from pathlib import Path

In [2]:
def make_sensory_regressors(exp, n_dirs=8, upsampling=5, sampling=1/3):
    
    stim = stim_vel_dir_dataframe(exp)
    bin_centres, dir_bins = quantize_directions(stim.theta)
    ind_regs = np.zeros((n_dirs, len(stim)))
    for i_dir in range(n_dirs):
        ind_regs[i_dir, :] = (np.abs(dir_bins - i_dir) < 0.1) & (stim.vel > 0.1)  

    dt_upsampled = sampling / upsampling
    t_imaging_up = np.arange(0, stim.t.values[-1], dt_upsampled)
    reg_up = interp1d(stim.t.values, ind_regs, axis=1, fill_value="extrapolate")(
        t_imaging_up
    )
    
    # 6s kernel
    u_steps = t_imaging_up.shape[0]
    u_time = np.arange(u_steps) * dt_upsampled
    decay = np.exp(-u_time / (1.5 / np.log(2)))
    kernel = decay / np.sum(decay)
    
    convolved = convolve2d(reg_up, kernel[None, :])[:, 0:u_steps]
    reg_sensory = convolved[:, ::upsampling]
    
    reg_up = reg_up[:, ::upsampling]

    return pd.DataFrame(reg_sensory.T, columns=[f"motion_{i}" for i in range(n_dirs)]), reg_up

In [3]:
master =  Path(r"/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data")
fish_list = list(master.rglob("*_f*"))
n_dirs = 8

In [4]:
for fish in fish_list:
    try:
        if not (fish / "sensory_regressors.h5").exists():
            print(fish)
            #for ls: 

            traces = fl.load(fish / "filtered_traces.h5", "/detr")
            len_rec = np.shape(traces)[0]
            print(len_rec)
            exp_ls = LightsheetExperiment(fish)
            fs = int(exp_ls.fn)
            sampling = 1/fs
            time = np.linspace(0, len_rec*sampling, len_rec)

            stim = stim_vel_dir_dataframe(exp_ls)

            theta = np.asarray(stim.theta)

            bin_centres, dir_bins = quantize_directions(stim.theta)
            ind_regs = np.zeros((n_dirs, len(stim)))
            for i_dir in range(n_dirs):
                ind_regs[i_dir, :] = (np.abs(dir_bins - i_dir) < 0.1) & (stim.vel > 0.1) 

            #len_rec, num_cells = np.shape(traces)
            # make a list of sensory regressors 
            reg, reg_interp = make_sensory_regressors(Experiment(fish), sampling=sampling)
            reg_list = [reg]
            print(np.shape(reg))
            print(len_rec)

            d = {
                'regressors': reg,
                'theta': theta,
                'individual_theta': ind_regs,
                'individual_theta_interp': reg_interp,
            }
            fl.save(fish / 'sensory_regressors.h5', d)
    except:
        print("Error")

/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data/suite2p_fast
Error
/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data/heading direction after habenula ablation/230530_f1
6660
Error
/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data/heading direction after habenula ablation/230607_f1
6660
Error
/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data/heading direction after habenula ablation/230607_f3
7200
Error
/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data/heading direction after habenula ablation/230530_f1/data_from_suite2p_unfiltered.h5
Error
/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data/heading direction after habenula ablation/230530_f1/anatomy_from_suite2p.tif
Error
/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data/hea